# Semana Data Science na Prática - QODA

## _Projeto 3 - Web Scraping no Instagram_ 📷

Nesse projeto, desenvolvi um programa que realiza um Web Scraping no Instagram, com o objetivo de encontrar e fazer o download de imagens que possuíssem a _hashtag_ #pizzasourdough. As imagens foram armazenadas no diretório _sourdoughpizzas_.

In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

### _Login_

In [3]:
# Especificando path para o chromedriver
driver = webdriver.Chrome(executable_path=r'C:\Users\gcout\projetos\web_scraping_instagram\chromedriver.exe')

# Abrindo o site
driver.get('http://www.instagram.com')

username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

# Entrada do username e senha
username.clear()
username.send_keys('gcoutos123')
password.clear()
password.send_keys('-')

# Encontra e clica no botão de login
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

### _Lidando com alertas_

In [4]:
time.sleep(5)
alert = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Agora não")]'))).click()
alert2 = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Agora não")]'))).click()

### _Buscando uma hashtag alvo_

In [5]:
# Encontra o campo de busca
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Pesquisar']")))
searchbox.clear()

# Busca pela hashtag sourdough
keyword = '#sourdoughpizza'
searchbox.send_keys(keyword)

# Concertando o double enter
time.sleep(5)
my_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/" + keyword[1:] + "/')]")))
my_link.click()

### _Scrolling down_

In [6]:
n_scrolls = 5

for j in range(0, n_scrolls):
    driver.execute_script('window.scrollTo(0, document.body.scrolHeight);')
    time.sleep(5)

In [7]:
# Encontra todos os link elements da página
anchors = driver.find_elements_by_tag_name('a')
anchors = [a.get_attribute('href') for a in anchors]

# Restringe todos os links para apenas links de imagem
anchors = [a for a in anchors if str(a). startswith('https://www.instagram.com/p/')]

print(f'Encontrei {str(len(anchors))} links de imagem!')
anchors[:5]

Encontrei 33 links de imagem!


['https://www.instagram.com/p/CM0GmFLnR71/',
 'https://www.instagram.com/p/CMzWGLvnQrK/',
 'https://www.instagram.com/p/CMDQtdgHhlu/',
 'https://www.instagram.com/p/CMUVJ3snj00/',
 'https://www.instagram.com/p/CM2ABMwH8er/']

In [8]:
images = []

# Segue cada link de imagem e extrai apenas imagens de index=1
for a in anchors:
    driver.get(a)
    time.sleep(5)
    img = driver.find_elements_by_tag_name('img')
    img = [i.get_attribute('src') for i in img]
    images.append(img[1])
    
    images[:5]

### _Salvando as imagens_

In [9]:
! pip install wget
import os
import wget

In [10]:
path = os.getcwd()
path = os.path.join(path, keyword[1:] + 's')

# Criando o diretório
os.mkdir(path)

In [11]:
# Dowload das imagens
counter = 0
for image in images:
    save_as = os.path.join(path, keyword[1:] + str(counter) + '.jpg')
    wget.download(image, save_as)
    counter += 1